In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df_mov = pd.read_csv( #"/datasets/movies/ml-20m/movies.csv")
                    "./movies_1m/ratings.dat", sep="::" , engine="python")
df_user_ratings = pd.read_csv(#"/datasets/movies/ml-20m/ratings.csv")
                        "./movies_1m/movies.dat", sep="::",engine="python" )

df_mov.set_index("movieId", drop=True)



df_ratings = df_user_ratings.merge(df_mov, on="movieId")

mov_ratings = df_ratings.groupby(by="movieId")[["rating"]].count()
mov_ratings["num_ratings"] = mov_ratings["rating"]
mov_ratings.drop(columns="rating", axis=1, inplace=True)

mov_ratings["avg_rating"] = df_ratings.groupby(by="movieId")["rating"].mean()


user_ratings = df_ratings.groupby(by="userId")[["rating"]].count()
user_ratings["num_user_ratings"] = user_ratings["rating"]
user_ratings.drop(columns="rating", axis=1, inplace=True)





df_full_ratings = pd.merge(df_ratings, mov_ratings, on="movieId")
df_full_ratings = pd.merge(df_full_ratings, user_ratings, on="userId")


min_ratings = 40
df_full_ratings = df_full_ratings[  df_full_ratings["num_ratings"] > min_ratings ]


df_mov = df_mov.merge( mov_ratings, how="inner", on="movieId")
df_mov = df_mov [ df_mov["num_ratings"] > min_ratings]
df_mov.set_index("movieId", drop=False, inplace=True)

user_mov_rat = pd.pivot_table(df_full_ratings , index="movieId", columns="userId", values="rating")

for col in user_mov_rat.columns:
    avg_rating = user_mov_rat[col].mean()
    user_mov_rat[col].fillna(2.5, inplace=True)



def to_movie_id(pivot_df, pivot_row_index):
    return pivot_df.index[pivot_row_index]

def to_pivot_row_index(pivot_df, movieId):
    index = pivot_df.index.to_list()
    i = 0

    j = len(index)
    while(i <= j):
        m = int((i + j)/2)
        if movieId < index[m]:
            j = m - 1
        elif movieId > index[m]:
            i = m + 1
        elif movieId == index[m]:
            return m
        
    return -1
        
def to_pivot_row_index_title(pivot_df, movie_df ,title):
    movieIdArray = movie_df[ movie_df["title"] == title ]["movieId"].values
    if len(movieIdArray) == 0:
        return None
    movieId = movieIdArray[0]
    return to_pivot_row_index(pivot_df, movieId)

def get_movieId_from_row_index(pivot_df, row_index):
    return pivot_df.index[row_index]

def get_movie_title_from_row_index(pivot_df, movie_df, row_index):
    movieId = pivot_df.index[row_index]
    return movie_df.loc[movieId]["title"]
    



movies_one_hot = pd.read_csv("/datasets/movies_1m/ml-1m/movies_one_hot.csv", index_col=0).drop(columns="title", axis=1)

#user_mov_rat = pd.merge(user_mov_rat, movies_one_hot, how="inner", on="movieId")
#print(user_mov_rat.head())




pairwise_similarities = cosine_similarity(user_mov_rat) 

movie_row_index = to_pivot_row_index_title(user_mov_rat, df_mov, 
                                                   "Jurassic Park (1993)"
                                                #"Air Force One (1997)"
                                                  # "Toy Story (1995)" 
                                          )

#print("row_index  ", movie_row_index)

similarities_for_movie =  np.round(pairwise_similarities[movie_row_index], 6)


# removed the selected movie from recommended list
sim_series = pd.Series(similarities_for_movie).drop(index=movie_row_index)

sim_series.sort_values(ascending=False, inplace=True)

for index in sim_series.index[:11]:
    print( get_movie_title_from_row_index(user_mov_rat, df_mov, index), "  ", sim_series[index]  )




In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df_mov = pd.read_csv( #"/datasets/movies/ml-20m/movies.csv")
                    "/datasets/movies_1m/ml-1m/movies.dat", sep="::", engine="python" )
df_user_ratings = pd.read_csv(#"/datasets/movies/ml-20m/ratings.csv")
                        "/datasets/movies_1m/ml-1m/ratings.dat", sep="::",  engine="python")

df_mov.set_index("movieId", drop=True)



df_ratings = df_user_ratings.merge(df_mov, on="movieId")

mov_ratings = df_ratings.groupby(by="movieId")[["rating"]].count()
mov_ratings["num_ratings"] = mov_ratings["rating"]
mov_ratings.drop(columns="rating", axis=1, inplace=True)

mov_ratings["avg_rating"] = df_ratings.groupby(by="movieId")["rating"].mean()


user_ratings = df_ratings.groupby(by="userId")[["rating"]].count()
user_ratings["num_user_ratings"] = user_ratings["rating"]
user_ratings.drop(columns="rating", axis=1, inplace=True)





df_full_ratings = pd.merge(df_ratings, mov_ratings, on="movieId")
df_full_ratings = pd.merge(df_full_ratings, user_ratings, on="userId")



min_ratings = 40
df_full_ratings = df_full_ratings[  df_full_ratings["num_ratings"] > min_ratings ]



df_mov = df_mov.merge( mov_ratings, how="inner", on="movieId")
df_mov = df_mov [ df_mov["num_ratings"] > min_ratings]
df_mov.set_index("movieId", drop=False, inplace=True)

user_mov_rat = pd.pivot_table(df_full_ratings , index="userId", columns="movieId", values="rating")



for col in user_mov_rat.columns:
    avg_rating = user_mov_rat[col].mean()
    user_mov_rat[col].fillna(2.5, inplace=True)
#'''


def to_movie_id(pivot_df, pivot_row_index):
    return pivot_df.index[pivot_row_index]

def to_pivot_row_index(pivot_df, movieId):
    index = pivot_df.index.to_list()
    i = 0

    j = len(index)
    while(i <= j):
        m = int((i + j)/2)
        if movieId < index[m]:
            j = m - 1
        elif movieId > index[m]:
            i = m + 1
        elif movieId == index[m]:
            return m
        
    return -1
        
def to_pivot_row_index_title(pivot_df, movie_df ,title):
    movieIdArray = movie_df[ movie_df["title"] == title ]["movieId"].values
    if len(movieIdArray) == 0:
        return None
    movieId = movieIdArray[0]
    return to_pivot_row_index(pivot_df, movieId)

def get_movieId_from_row_index(pivot_df, row_index):
    return pivot_df.index[row_index]

def get_movie_title_from_row_index(pivot_df, movie_df, row_index):
    movieId = pivot_df.index[row_index]
    return movie_df.loc[movieId]["title"]
   
    

title = "Jurassic Park (1993)"

movie_row = df_mov[df_mov["title"] == title ]
movieId = movie_row.index[0]


chosen_movie_ratings = user_mov_rat[movieId]

corr_with_chosen_movie = np.round(user_mov_rat.corrwith(chosen_movie_ratings, axis=0), 3)
corr_with_chosen_movie.drop(index=movieId, inplace=True)


top_n_with_corr = corr_with_chosen_movie.sort_values(ascending=False)[:10] 


for movieId in top_n_with_corr.index:
     title = df_mov.loc[movieId]["title"]
     print(title, " : ", top_n_with_corr.loc[movieId] )



In [ ]:

from sklearn.decomposition import NMF




# usare df_full_ratings
user_mov_rat = pd.pivot_table(df_full_ratings , index="userId", columns="movieId", values="rating")
for col in user_mov_rat.columns:
    avg_rating = user_mov_rat[col].mean()
    user_mov_rat[col].fillna(2.5, inplace=True)

    
    
    
    
user_mov_rat_with_nans = pd.pivot_table(df_full_ratings , index="userId", columns="movieId", values="rating")


def get_userId_from_index(user_row_index, user_mov_rat_with_nans):
    return user_mov_rat_with_nans.index[user_row_index]
def get_movieId_from_index(movie_col_index, user_mov_rat_with_nans):
    return user_mov_rat_with_nans.columns[movie_col_index]

def get_title_from_movieId(movieId, movies_df):
    return movies_df.loc[movieId]["title"]


def get_seen_for_user_index(user_mov_rat_with_nans, user_row_index, movies_df):
    userId = get_userId_from_index(user_row_index, user_mov_rat_with_nans)
    
  
    best_for_user = user_mov_rat_with_nans.loc[userId].sort_values(ascending=False)
    #print("best for user ")
   
    seen = []
    for movieId in  best_for_user.index:
        if np.isnan(best_for_user[movieId]) == False:
            seen.append( (movieId, movies_df.loc[movieId]["title"],    best_for_user[movieId] )    )
         
    
    #print(seen)
    return seen


def get_unseen_movie_ids( user_mov_rat_with_nans, user_row_index ):
    
    user_series = user_mov_rat_with_nans.loc[user_row_index]
    user_not_seen = user_series[ user_series.isna() ]
    return user_not_seen.index.to_list()
 



user_index = 0


R = user_mov_rat.values
model = NMF(n_components=150, init='random', random_state=0)


W = model.fit_transform(R)
H = model.components_


predictions =  np.round(W.dot(H), 3 ) 

top_n_movies_for_user = []

predicted_for_user = predictions[user_index]
sorted_movies_pos = np.argsort(-predicted_for_user)


i = 1

userId = get_userId_from_index(user_index, user_mov_rat_with_nans)
seen_movies = list(map(lambda x : x[0]  ,get_seen_for_user_index(user_mov_rat_with_nans, user_index, df_mov)  )  )

print("Best seen")
for movieId, title, rating in get_seen_for_user_index(user_mov_rat_with_nans, user_index, df_mov)[:20]:
    print(title, "  ", rating)
print()
print("Recommended")

pred_series = pd.Series( predicted_for_user ).sort_values(ascending=False)


i = 1
for movie_col_index, pred_rating in zip(pred_series.index.to_list(), pred_series.values):
    movieId = get_movieId_from_index(movie_col_index, user_mov_rat_with_nans)
    if movieId not in seen_movies:
            print(i, " : ", get_title_from_movieId(movieId, df_mov), " ", movieId, " ", pred_rating )
            i += 1
            if i > 10:
                break


mse = 0.0

R_with_zeros = user_mov_rat_with_nans.fillna(0).values

xs, ys = R_with_zeros.nonzero()
predictions =  W.dot(H)
count = 0
for x_test, y_test in zip( xs, ys ):
    mse += pow(R_with_zeros[x_test, y_test] - predictions[x_test, y_test], 2)
    count += 1
rmse = np.sqrt(mse/count)
print(rmse)











